In [1]:
import orjson

from conda_oci_utils import (
    encode_conda_dist_to_oci_dist,
    is_valid_oci_dist,
    is_valid_conda_dist,
    VALID_NAME_RE,
    VALID_TAG_RE,
)
from conda_forge_metadata.repodata import fetch_repodata, all_labels

In [2]:
import tqdm

labels = all_labels(use_remote_cache=True)

max_name_len = 0
max_tag_len = 0
for label in tqdm.tqdm(labels, ncols=80):
    for subdir in [
        "linux-64",
        "osx-64",
        "noarch",
        "win-64",
        "linux-aarch64",
        "osx-arm64",
        "linux-ppc64le",
    ]:
        rd = fetch_repodata([subdir], label=label)[0]
        with open(rd) as f:
            rd = orjson.loads(f.read())

        for key in ["packages", "packages.conda"]:
            for dist in rd[key]:
                if not is_valid_conda_dist(dist):
                    tqdm.tqdm.write(f"invalid conda dist: {dist} (label {label})")

                if label != "main":
                    conda_dist = f"conda-forge/{label}/{subdir}/{dist}"
                else:
                    conda_dist = f"conda-forge/{subdir}/{dist}"

                if not is_valid_conda_dist(conda_dist):
                    tqdm.tqdm.write(f"invalid conda dist: {conda_dist} (label: {label})")

                oci_dist = encode_conda_dist_to_oci_dist(conda_dist)

                if not is_valid_oci_dist(oci_dist):
                    tqdm.tqdm.write(f"invalid oci dist: {oci_dist} (label: {label})")
                else:
                    name, tag = oci_dist.rsplit(":", maxsplit=1)
                    # if len(name) > max_name_len:
                    #     tqdm.tqdm.write(f"current maximum oci name length: {len(name)}")
                    max_name_len = max(max_name_len, len(name))
                    # if len(tag) > max_tag_len:
                    #     tqdm.tqdm.write(f"current maximum oci tag length: {len(tag)}")
                    max_tag_len = max(max_tag_len, len(tag))
                    # tqdm.tqdm.write(f"valid oci dist: {oci_dist} (label: {label})")

print(f"max oci name length: {max_name_len}")
print(f"max oci tag length: {max_tag_len}")

  7%|██▊                                      | 25/357 [00:00<00:02, 132.83it/s]

invalid conda dist: pyside2-2.0.0~alpha0-py27_0.tar.bz2 (label broken-and-invalid)
invalid conda dist: conda-forge/broken-and-invalid/linux-64/pyside2-2.0.0~alpha0-py27_0.tar.bz2 (label: broken-and-invalid)
invalid oci dist: conda-forge/linux-64/cpyside2:broken-and-invalid-2.0.0~alpha0-py27__0 (label: broken-and-invalid)
invalid conda dist: pyside2-2.0.0~alpha0-py35_0.tar.bz2 (label broken-and-invalid)
invalid conda dist: conda-forge/broken-and-invalid/linux-64/pyside2-2.0.0~alpha0-py35_0.tar.bz2 (label: broken-and-invalid)
invalid oci dist: conda-forge/linux-64/cpyside2:broken-and-invalid-2.0.0~alpha0-py35__0 (label: broken-and-invalid)
invalid conda dist: pyside2-2.0.0~alpha0-py36_0.tar.bz2 (label broken-and-invalid)
invalid conda dist: conda-forge/broken-and-invalid/linux-64/pyside2-2.0.0~alpha0-py36_0.tar.bz2 (label: broken-and-invalid)
invalid oci dist: conda-forge/linux-64/cpyside2:broken-and-invalid-2.0.0~alpha0-py36__0 (label: broken-and-invalid)
invalid conda dist: pyside2-2.0

100%|█████████████████████████████████████████| 357/357 [00:25<00:00, 14.24it/s]

max oci name length: 71
max oci tag length: 88


In [3]:
import numpy as np

labels = all_labels(use_remote_cache=True)

chars = set()
for label in labels:
    if not VALID_NAME_RE.match(label):
        print(f"non-oci name label: {label}")
    if not VALID_TAG_RE.match(label):
        print(f"non-oci tag label: {label}")
    for lc in label:
        chars.add(lc)

print("max label length:", max(len(label) for label in labels))
print("unique label characters:", sorted(chars))
print("max length label:", labels[np.argmax([len(label) for label in labels])])

non-oci name label: TEST
max label length: 35
unique label characters: ['-', '.', '0', '1', '2', '3', '4', '5', '6', '7', '8', '9', 'E', 'S', 'T', '_', 'a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 'k', 'l', 'm', 'n', 'o', 'p', 'q', 'r', 's', 't', 'u', 'v', 'w', 'x', 'y', 'z']
max length label: robotframework_jupyterlibrary_alpha
